In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLYJ931/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLYJ931/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLYJ931/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
!python --version

Python 3.6.9


In [32]:
train_data, test_data = imdb["train"], imdb["test"]
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for sent,label in train_data:
  training_sentences.append(str(sent.numpy()))
  training_labels.append(label.numpy())

for sent, label in test_data:
  testing_sentences.append(str(sent.numpy()))
  testing_labels.append(label.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

print(training_labels[:5])
print(testing_labels[:5])

[0, 0, 0, 1, 1]
[1, 1, 0, 0, 1]


In [16]:
# declaring Hyperparameters
vocab_size = 15000
embedding_size = 16
max_length = 150
trunc_type = "post"
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
train_word_index = tokenizer.word_index
training_seq = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(training_seq, maxlen = max_length, truncating = trunc_type)


testing_seq = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_seq, maxlen = max_length, truncating = trunc_type)
print(padded)
print(testing_padded)



[[   0    0    0 ...  874  145   10]
 [   0    0    0 ...   32   31   47]
 [   0    0    0 ...    9 6176   47]
 ...
 [7629   37   11 ... 7634  375 7386]
 [2677   11  216 ...  182    6  728]
 [3875    5   31 ...   21   24    2]]
[[  59   44   25 ...    5    2  122]
 [  59    4    1 ...  283   16  114]
 [   1   18  301 ...   12   20   94]
 ...
 [  59   92    1 ...   70   60    3]
 [   0    0    0 ...   54   74  220]
 [   0    0    0 ...  458 8360 2149]]


In [17]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_size, input_length = max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation= 'relu'),
                             tf.keras.layers.Dense(1, activation= 'sigmoid')
                             
])
model.compile(loss = "binary_crossentropy", optimizer ="adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           240000    
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 14406     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 254,413
Trainable params: 254,413
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 15
model.fit(padded, training_labels_final, epochs = num_epochs, validation_data= (testing_padded, testing_labels_final) )

Epoch 1/15
782/782 [==============================] - 5s 6ms/step - loss: 5.0334e-05 - accuracy: 1.0000 - val_loss: 0.8925 - val_accuracy: 0.8241
Epoch 2/15
782/782 [==============================] - 5s 6ms/step - loss: 3.1218e-05 - accuracy: 1.0000 - val_loss: 0.9261 - val_accuracy: 0.8241
Epoch 3/15
782/782 [==============================] - 5s 6ms/step - loss: 1.9803e-05 - accuracy: 1.0000 - val_loss: 0.9644 - val_accuracy: 0.8235
Epoch 4/15
782/782 [==============================] - 5s 6ms/step - loss: 1.2412e-05 - accuracy: 1.0000 - val_loss: 0.9963 - val_accuracy: 0.8241
Epoch 5/15
782/782 [==============================] - 5s 6ms/step - loss: 7.8675e-06 - accuracy: 1.0000 - val_loss: 1.0321 - val_accuracy: 0.8238
Epoch 6/15
782/782 [==============================] - 5s 6ms/step - loss: 5.0072e-06 - accuracy: 1.0000 - val_loss: 1.0656 - val_accuracy: 0.8239
Epoch 7/15
782/782 [==============================] - 5s 6ms/step - loss: 3.2362e-06 - accuracy: 1.0000 - val_loss: 1.0998 -

In [22]:
# let's create another model with different flatten layer
new_model = tf.keras.Sequential([
                   tf.keras.layers.Embedding(vocab_size, embedding_size, input_length= max_length),
                   tf.keras.layers.GlobalAveragePooling1D(),
                   tf.keras.layers.Dense(6, 'relu'),
                   tf.keras.layers.Dense(1, 'sigmoid')
])
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 16)           240000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
Total params: 240,109
Trainable params: 240,109
Non-trainable params: 0
_________________________________________________________________


In [23]:
num_epochs = 15
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/15
782/782 [==============================] - 5s 7ms/step - loss: 7.6987e-08 - accuracy: 1.0000 - val_loss: 1.3917 - val_accuracy: 0.8226
Epoch 2/15
782/782 [==============================] - 5s 7ms/step - loss: 5.3599e-08 - accuracy: 1.0000 - val_loss: 1.4213 - val_accuracy: 0.8226
Epoch 3/15
782/782 [==============================] - 5s 7ms/step - loss: 3.7671e-08 - accuracy: 1.0000 - val_loss: 1.4473 - val_accuracy: 0.8224
Epoch 4/15
782/782 [==============================] - 5s 6ms/step - loss: 2.7092e-08 - accuracy: 1.0000 - val_loss: 1.4771 - val_accuracy: 0.8222
Epoch 5/15
782/782 [==============================] - 5s 6ms/step - loss: 1.9879e-08 - accuracy: 1.0000 - val_loss: 1.5002 - val_accuracy: 0.8222
Epoch 6/15
782/782 [==============================] - 5s 6ms/step - loss: 1.4660e-08 - accuracy: 1.0000 - val_loss: 1.5254 - val_accuracy: 0.8222
Epoch 7/15
782/782 [==============================] - 5s 7ms/step - loss: 1.1188e-08 - accuracy: 1.0000 - val_loss: 1.5523 -

In [28]:
e_layer = model.layers[0]
weights = e_layer.get_weights()[0]
print(weights.shape)

(15000, 16)


In [29]:
new_elayer = model.layers[0]
new_weights = new_elayer.get_weights()[0]
print(new_weights.shape)

(15000, 16)


In [31]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]
